In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pendulum
from tqdm import trange

def init_spark(app_name: str = "default_test_sparkapp"):
    spark_conf = SparkConf()
    spark_conf.setAppName(app_name)
    spark = SparkSession.builder.config(conf=spark_conf).enableHiveSupport().getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    spark.sql("set hive.exec.dynamic.partition.mode = nonstrict")
    spark.sql("set hive.exec.dynamic.partition=true")
    return spark

spark = init_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/05 07:58:20 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/05 07:58:27 WARN lineage.LineageWriter: Lineage directory /var/log/spark/lineage doesn't exist or is not writable. Lineage for this application will be disabled.
23/05/05 07:58:27 WARN lineage.LineageWriter: Lineage directory /var/log/spark/lineage doesn't exist or is not writable. Lineage for this application will be disabled.


In [2]:
feature_name = [
    "acc_cost_usd",
    "acc_first_purchase_price_usd",
    "acc_role_cnt",
    "acc_total_charge_cnt",
    "acc_total_charge_sum_usd",
    "acc_total_refund_cnt",
    "acc_total_refund_sum_usd",
    "bundle_pop_buy_count_last_day",
    "bundle_pop_buy_count_last_day_campaign_stage_trigger",
    "bundle_pop_buy_count_last_day_tech_unlock_trigger",
    "bundle_pop_buy_count_last_week",
    "bundle_pop_buy_count_last_week_adjust",
    "bundle_pop_buy_max_last_day",
    "bundle_pop_buy_max_last_week",
    "bundle_pop_buy_max_last_week_adjust",
    "bundle_pop_buy_max_price_campaign_stage_trigger",
    "bundle_pop_buy_max_price_campaign_stage_trigger_commax",
    "bundle_pop_buy_max_price_tech_unlock_trigger",
    "bundle_pop_buy_max_price_tech_unlock_trigger_commax",
    "bundle_pop_buy_sum_last_day",
    "bundle_pop_buy_sum_last_week",
    "bundle_pop_buy_sum_last_week_adjust",
    "bundle_pop_count_campaign_stage_trigger_9999",
    "bundle_pop_count_last_day",
    "bundle_pop_count_last_day_campaign_stage_trigger",
    "bundle_pop_count_last_day_tech_unlock_trigger",
    "bundle_pop_count_last_week",
    "bundle_pop_count_last_week_adjust",
    "bundle_pop_count_tech_unlock_trigger_9999",
    "buy_rate_last_day",
    "buy_rate_last_day_campaign_stage_trigger",
    "buy_rate_last_day_tech_unlock_trigger",
    "buy_rate_last_week",
    "if_accounts_first_role",
    "if_pruchase_active_in_1day",
    "if_pruchase_active_in_30day",
    "if_pruchase_active_in_3day",
    "if_pruchase_active_in_7day",
    "is_acc_transfer_from_old",
    "is_acc_transfer_to_new",
    "is_role_transfer_from_old",
    "is_role_transfer_to_new",
    "is_valid",
    "level",
    "max_role_level",
    "money_flow_in_cout_last_day",
    "money_flow_in_cout_last_week",
    "money_flow_in_cout_last_week_adjust",
    "money_flow_out_cout_last_day",
    "money_flow_out_cout_last_week",
    "money_flow_out_cout_last_week_adjust",
    "purchase_price_1499_last_week",
    "purchase_price_1499_last_week_adjust",
    "purchase_price_1999_last_week",
    "purchase_price_1999_last_week_adjust",
    "purchase_price_2999_last_week",
    "purchase_price_2999_last_week_adjust",
    "purchase_price_4999_last_week",
    "purchase_price_4999_last_week_adjust",
    "purchase_price_499_last_week",
    "purchase_price_499_last_week_adjust",
    "purchase_price_9999_last_week",
    "purchase_price_9999_last_week_adjust",
    "purchase_price_999_last_week",
    "purchase_price_999_last_week_adjust",
    "purchase_price_max",
    "purchase_price_max_last_3days",
    "purchase_price_max_last_day",
    "purchase_price_max_last_month",
    "purchase_price_max_last_week",
    "purchase_price_max_last_week_adjust",
    "purchase_price_sum_last_3days",
    "purchase_price_sum_last_day",
    "purchase_price_sum_last_month",
    "purchase_price_sum_last_week",
    "purchase_price_sum_last_week_adjust",
    "purchase_price_sum_max_ratio",
    "purchase_price_sum_max_ratio_com2",
    "role_cost_usd",
    "role_create_os_android",
    "role_create_os_ios",
    "role_first_purchase_price_usd",
    "role_last_login_duration",
    "role_last_purchase_price_usd",
    "role_lifespan",
    "role_login_last_now",
    "role_login_secondlast_last",
    "role_login_secondlast_now",
    "role_logout_last_now",
    "role_no",
    "role_purchase_first_last",
    "role_purchase_first_now",
    "role_purchase_last_now",
    "role_total_charge_cnt",
    "role_total_charge_sum_usd",
    "role_total_refund_cnt",
    "role_total_refund_sum_usd",
    "role_type_fraud",
    "role_type_no_install_info",
    "vip_level",
]

s = "column_name : {}\n\n".format(",".join(feature_name))
for slot, x in enumerate(feature_name):
    s += f"feature_name=r_{x};depend={x};method=Float;slot={slot};args=-100\n"


In [7]:
originaldt = pendulum.Date(2023, 4, 1)
extractUDF = F.udf(
    lambda x: "\x02".join(
        [v for v in x.split("\x02") if v.split("\x03")[0] in feature_name]
    ),
    T.StringType(),
)

# same as 220627
for i in trange(0, 30, 2):
    dt = originaldt.add(days=i)
    dtstr = dt.to_date_string()
    start, end = dt.add(days=1).to_date_string(), dt.add(days=7).to_date_string()

    if end >= pendulum.today().to_date_string():
        break

    label = (
        spark.sql(
            f"select * from 10048_oss_bi_dw.wgame_trigger_bundle_v2 where dt between '{start}' and '{end}' and action = 'spc_bundle_buy' "
        )
        .dropDuplicates(
            ["open_id", "user_id", "trace_id", "trigger_type", "bundle_price"]
        )
        .groupBy(["open_id", "user_id", "trigger_type"])
        .agg(F.max(F.col("bundle_price").cast(T.FloatType())).alias("bundle_price"))
        .withColumn("if_buy_pop_gift", F.lit(1))
        .withColumnRenamed("open_id", "acc_id")
        .withColumnRenamed("user_id", "role_id")
    )

    sample = (
        spark.sql(
            f"select * from 10048_ml_dw.bundle_price_role_common_feature_dd where dt = '{dtstr}' and  feature_group = 'attribute' "
        )
        .withColumn("subf", F.explode(F.split(F.col("feature"), "\x02")))
        .withColumn("fname", F.substring_index(F.col("subf"), "\x03", 1))
        .where(F.col("fname").isin(feature_name))
        .groupBy("acc_id", "role_id", "dt")
        .agg(F.concat_ws("\x02", F.collect_set("subf")).alias("feature"))
        .join(label, on=["acc_id", "role_id"], how="inner")
        .withColumn("dt", F.col("dt"))
        .withColumn("grp", F.lit("maxp7day"))
        .select(
            "acc_id",
            "role_id",
            "feature",
            "trigger_type",
            "bundle_price",
            "if_buy_pop_gift",
            "dt",
            "grp",
        )
    )

    sample.write.mode("overwrite").insertInto(
        "ml_tmp_db.training_sample_for_wgame_bundle_price_002", overwrite=True
    )

    sample.unpersist()
    label.unpersist()


  0%|          | 0/15 [00:00<?, ?it/s]setfacl: Permission denied. user=biuser is not the owner of inode=/user/hive/warehouse/ml_tmp_db.db/training_sample_for_wgame_bundle_price_002/.hive-staging_hive_2023-05-04_10-17-26_768_804367023233551585-1
setfacl: Permission denied. user=biuser is not the owner of inode=/user/hive/warehouse/ml_tmp_db.db/training_sample_for_wgame_bundle_price_002/dt=2023-04-01
setfacl: Permission denied. user=biuser is not the owner of inode=/user/hive/warehouse/ml_tmp_db.db/training_sample_for_wgame_bundle_price_002/dt=2023-04-01/grp=maxp7day
  7%|▋         | 1/15 [03:31<49:17, 211.28s/it]setfacl: Permission denied. user=biuser is not the owner of inode=/user/hive/warehouse/ml_tmp_db.db/training_sample_for_wgame_bundle_price_002/.hive-staging_hive_2023-05-04_10-20-57_789_7481603083111023431-1
setfacl: Permission denied. user=biuser is not the owner of inode=/user/hive/warehouse/ml_tmp_db.db/training_sample_for_wgame_bundle_price_002/dt=2023-04-03
setfacl: Permiss

23/05/04 10:44:10 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /10.136.220.159:34220 is closed
23/05/04 10:44:10 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /10.136.220.159:34528 is closed


In [32]:
dt = pendulum.Date(2023,5,1)
dtstr = dt.to_date_string()
start, end = dt.add(days=1).to_date_string(), dt.add(days=7).to_date_string()
extractUDF = F.udf(lambda x: '\x02'.join([v for v in x.split('\x02') if v.split('\x03')[0] in feature_name]), T.StringType())


label = (
    spark
    .sql(f"select * from 10048_oss_bi_dw.wgame_trigger_bundle_v2 where dt between '{start}' and '{end}' and action = 'spc_bundle_buy' ")
    .dropDuplicates(['open_id', 'user_id', 'trace_id', 'trigger_type', 'bundle_price']) 
    .groupBy(["open_id","user_id","trigger_type"]).agg(F.max(F.col("bundle_price").cast(T.FloatType())).alias("bundle_price"))
    .withColumn('if_buy_pop_gift', F.lit(1))
    .withColumnRenamed('open_id','acc_id')
    .withColumnRenamed('user_id','role_id')
)


sample = (
    spark
    .sql(f"select * from 10048_ml_dw.bundle_price_role_common_feature_dd where dt = '{dtstr}' and  feature_group = 'attribute' ")
    .withColumn('subf', F.explode(F.split(F.col('feature'),'\x02')))
    .withColumn('fname', F.substring_index(F.col('subf'),'\x03', 1))
    .where(F.col('fname').isin(feature_name))
    .groupBy('acc_id', 'role_id', 'dt').agg( F.concat_ws('\x02', F.collect_set('subf')).alias('feature'))
    .join(label, on = ['acc_id','role_id'],how='inner')
    .withColumn('dt', F.col('dt'))
    .withColumn('grp', F.lit('maxp7day'))
    .select('acc_id','role_id','feature','trigger_type','bundle_price', 'if_buy_pop_gift','dt','grp')
)
sample.printSchema()
# sample.write.mode("overwrite").insertInto('ml_tmp_db.training_sample_for_wgame_bundle_price_002', overwrite=True)

root
 |-- acc_id: string (nullable = true)
 |-- role_id: string (nullable = true)
 |-- feature: string (nullable = false)
 |-- trigger_type: string (nullable = true)
 |-- bundle_price: float (nullable = true)
 |-- if_buy_pop_gift: integer (nullable = false)
 |-- dt: string (nullable = true)
 |-- grp: string (nullable = false)



In [ ]:
# CREATE TABLE IF NOT EXISTS hive.ml_tmp_db.training_sample_for_wgame_bundle_price_002 (
#     acc_id varchar,
#     role_id varchar, 
#     feature array(map(varchar,real)),
#     trigger_type varchar,
#   	bundle_price real,
#   	if_buy_pop_gift integer,
#     dt varchar, 
#     grp varchar
# ) WITH (
#     FORMAT = 'PARQUET',
#     partitioned_by = ARRAY['dt', 'grp']
# )

root
 |-- acc_id: string (nullable = true)
 |-- role_id: string (nullable = true)
 |-- feature: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: float (valueContainsNull = true)
 |-- bundle_price: float (nullable = true)
 |-- dt: string (nullable = true)
 |-- grp: string (nullable = false)

